<a href="https://colab.research.google.com/github/npwitk/NLP1001-Introduction-to-NLP/blob/main/POS_tagging_DL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Change hardware to GPU first

In [2]:
import nltk
nltk.download("treebank")
tagged_sentences = nltk.corpus.treebank.tagged_sents()

print(tagged_sentences[0])
print("Tagged sentences: ", len(tagged_sentences))
print("Tagged words: ", len(nltk.corpus.treebank.tagged_words()))

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
Tagged sentences:  3914
Tagged words:  100676


In [3]:
import numpy as np

sentences, sentence_tags = [], [] # แยกออกมาเป็น List ของคำ และ List ของ Tag

for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append(np.array(sentence))
    sentence_tags.append(np.array(tags))

# Let's see how a sequence looks

print(sentences[5])
print(sentence_tags[5])

['Lorillard' 'Inc.' ',' 'the' 'unit' 'of' 'New' 'York-based' 'Loews'
 'Corp.' 'that' '*T*-2' 'makes' 'Kent' 'cigarettes' ',' 'stopped' 'using'
 'crocidolite' 'in' 'its' 'Micronite' 'cigarette' 'filters' 'in' '1956'
 '.']
['NNP' 'NNP' ',' 'DT' 'NN' 'IN' 'JJ' 'JJ' 'NNP' 'NNP' 'WDT' '-NONE-' 'VBZ'
 'NNP' 'NNS' ',' 'VBD' 'VBG' 'NN' 'IN' 'PRP$' 'NN' 'NN' 'NNS' 'IN' 'CD'
 '.']


In [4]:
from sklearn.model_selection import train_test_split

(train_sentences, test_sentences, train_tags, test_tags) = train_test_split(sentences, sentence_tags, test_size = 0.2)

Keras also needs to work with numbers, (Map คำให้อยู่ในรูปตัวเลขก่อน)

In [5]:
words, tags = set([]), set([])

for s in train_sentences:
    for w in s:
        words.add(w.lower())

for ts in train_tags:
    for t in ts:
        tags.add(t)

word2index = {w: i + 2 for i, w in enumerate(list(words))}
word2index['-PAD-'] = 0 # The special value used for padding
word2index['-OOV-'] = 1 # The special value used of OOVs (Out of vocab, ให้ model ทนทานมากขึ้น)

tag2index = {t: i + 1 for i, t in enumerate(list(tags))}
tag2index['-PAD-'] = 0 # The special value used for padding

In [6]:
train_sentences_X, test_sentences_X, train_tags_y, test_tags_y = [], [], [], []

for s in train_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    train_sentences_X.append(s_int)

for s in test_sentences:
    s_int = []
    for w in s:
        try:
            s_int.append(word2index[w.lower()])
        except KeyError:
            s_int.append(word2index['-OOV-'])

    test_sentences_X.append(s_int)

for s in train_tags:
    train_tags_y.append([tag2index[t] for t in s])

for s in test_tags:
    test_tags_y.append([tag2index[t] for t in s])

print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[5648, 6288, 9597, 1890, 1414, 9441, 9087, 8293, 344, 9462, 5634, 4956, 4883, 4956, 5243, 5175]
[9855, 5150, 9228, 6480, 6532, 1953, 2721, 6452, 6839, 9667, 344, 3289, 3872, 9851, 8800, 6243, 5668, 9000, 9764, 10001, 5852, 3238, 2915, 5175]
[42, 42, 37, 22, 14, 27, 30, 4, 27, 40, 11, 9, 11, 9, 30, 15]
[42, 32, 28, 25, 11, 11, 22, 22, 16, 28, 27, 30, 40, 22, 14, 25, 27, 11, 40, 22, 14, 27, 30, 15]


In [7]:
MAX_LEGTH = len(max(train_sentences_X, key=len))
print(MAX_LEGTH)

271


In [8]:
from tensorflow.keras.preprocessing.sequence import pad_sequences # ตัวไหนที่ len ไม่ถึงก็แปะ 0 เข้าท้าย

train_sentences_X = pad_sequences(train_sentences_X, maxlen=MAX_LEGTH, padding='post')
test_sentences_X = pad_sequences(test_sentences_X, maxlen=MAX_LEGTH, padding='post')
train_tags_y = pad_sequences(train_tags_y, maxlen=MAX_LEGTH, padding='post')
test_tags_y = pad_sequences(test_tags_y, maxlen=MAX_LEGTH, padding='post')

print(train_sentences_X[0])
print(test_sentences_X[0])
print(train_tags_y[0])
print(test_tags_y[0])

[5648 6288 9597 1890 1414 9441 9087 8293  344 9462 5634 4956 4883 4956
 5243 5175    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0 

In [ ]:
!pip install tensorflow
!pip install keras

In [13]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, InputLayer, Bidirectional, TimeDistributed, Embedding, Activation
from tensorflow.keras.optimizers import Adam

model = Sequential()
model.add(InputLayer(input_shape=(MAX_LEGTH, )))
model.add(Embedding(len(word2index), 128))
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(len(tag2index))))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 271, 128)            │       1,287,296 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_2 (Bidirectional)      │ (None, 271, 512)            │         788,480 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ time_distributed_2 (TimeDistributed) │ (None, 271, 47)             │          24,111 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 271, 47)             │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,099,887 (8.01 MB)

 Trainable params: 2,099,887 (8.01 MB)

 Non-trainable params: 0 (0.00 B)

One-Hot Encoded tags.

In [14]:
def to_categorical(sequences, categories):
    cat_sequences = []
    for s in sequences:
        cats = []
        for item in s:
            cats.append(np.zeros(categories))
            cats[-1][item] = 1.0
        cat_sequences.append(cats)
    return np.array(cat_sequences)

In [15]:
cat_train_tags_y = to_categorical(train_tags_y, len(tag2index))
print(cat_train_tags_y[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]]


In [ ]:
model.fit(train_sentences_X, to_categorical(train_tags_y, len(tag2index)), batch_size=128, epochs=40, validation_split=0.2)

Epoch 1/40


In [ ]:
scores = model.evaluate(test_sentences_X, to_categorical(test_tags_y, len(tag2index)))
print(f"{model.metrics_names[1]} : {scores[1] == 100}")

In [ ]:
test_samples = [
    "Running is very important for me".split(),
    "I was running every day for a month".split()
]

print(test_samples)

In [ ]:
test_samples_X = []
for s in test_samples:
  s_int = []
  for w in s:
    try:
      s_int.append(word2index[w.lower()])
    except KeyError:
      s_int.append(word2index['-OOV-'])
  test_samples_X.append(s_int)

test_samples_X = pad_sequences(test_samples_X, maxlen = MAX_LENGTH, padding = 'post')
print(test_samples_X)

In [ ]:
predictions = model.predict(test_samples_X)
print(predictions, predictions.shape)

In [ ]:
def logits_to_tokens(sequences, index):
  token_sequences = []
  for categorical_sequence in sequences:
    token_sequence = []
    for categorical in categorical_sequence:
      token_sequence.append(index[np.argmax(categorical)])

    token_sequences.append(token_sequence)

  return token_sequences

In [ ]:
print(logits_to_tokens(predictions, {i: t for t, i in tag2index.items()}))